# Trabajo Final procesamiento de lenguaje natural (NLP) <a class="tocSkip">
## Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** DANIEL GÓNGORA GARCIA

**ID:** 000571107


**Subir Este notebook de Trabajo Final en su repositorio de GitHub. y enviar el link en **Microsoft TEAMS**

Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo
Realizar un proceso de Procesamiento de lenguaje natural (NLP) para dejar los datos preparados para ser usados con algoritmos de Machine Learning para Clasificación como objetivo final del trabajo.

los datos preparados para ser usados con algoritmos de Machine Learning para Regresión o Clasificación como objetivo final del trabajo.

El trabajo se realizara en este jupyter notebook y subirlo a su repositorio de github creado en clase. (**Recuerde poner su nombre e información**)

## Las actividades a realizar
    

1) Limpiar los datos de texto.

   - https://joserzapata.github.io/courses/nlp/procesamiento-basico/

   - https://joserzapata.github.io/courses/nlp/preprocesamiento-texto/

2) Realizar la representación de texto:

   - https://joserzapata.github.io/courses/nlp/representaciones/

   - Tokenización
   - Lematización o stemming
   - Representación de los datos de texto (Bag of Words o TF-IDF)

3) Utilice un modelo de Machine Learning para clasificación para entrenar y evaluar el modelo.
        
    - Calcule estas las métricas de evaluación (accuracy, precision, recall, f1-score) - https://joserzapata.github.io/courses/python-ciencia-datos/clasificacion/#evaluacion-modelo-simple


    
   
    
*NOTA: No dude en contactarme para cualquier pregunta o inquietud :) por el chat de Teams o al correo
joser.zapata@upb.edu.co*

## EVALUACIÓN


|Porcentaje en la evaluación | Descripción| Nada | Incompleto | Completo 
| :---: |:---: |:---: |:---: |:---: 
| 5 % |**limpieza de los datos** |
| 5 % | **Representacion de los datos** |
| 5 % | **Machine Learning** <br> Entrenar y evaluar el modelo propuesto |


In [47]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk

# Descargar stopwords en español
#nltk.download("stopwords")
from nltk.corpus import stopwords

# Obtener stopwords en español
stop_words = set(stopwords.words("spanish"))

Carga del dataset

El archivo  se carga en aproximadamente 10 segundos

In [7]:
# Usar su ID UPB Ejemplo: "0028984798"
id_upb = "000571107"

data_reviews = pd.read_parquet(
    "https://www.dropbox.com/scl/fi/gvk9yj8cn96oocr9z058x/filmaffinity_reviews_dataset.parquet?rlkey=xgvr00zvkxbkwqqavqutpsshg&st=xjb7xze9&dl=1",
    engine="fastparquet",
)
data_reviews = data_reviews.sample(n=50_000, random_state=int(id_upb))
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 16544 to 4695
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author_review_desc  50000 non-null  object
 1   author_rating       50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


Ejemplo de algunas filas del dataset

In [8]:
data_reviews.sample(5)

,author_review_desc,author_rating
20995,"\nBajo mi humilde punto de vista, esta pelicul...",9
96350,\nCon el título ya he hecho un resumen de la p...,1
88208,\nLa película narra la historia del verano de ...,7
88020,\nHicieron la película sin un asesor colombian...,3
129797,\n¨El imperio de los cadáveres¨ es un popurrí ...,6


Evaluar los valores nulos

In [9]:
data_reviews.isnull().sum()

author_review_desc    0
author_rating         0
dtype: int64

En este caso no hay nulos.

Revisar si hay valores duplicados:

In [10]:
data_reviews.duplicated().sum()

np.int64(2)

No hay  valores duplicados.

## Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎



Definición: Se considera reseña positiva cuando la puntuación ("author_rating") es mayor que 6; negativa en caso contrario.

Crear la variable binaria de sentimiento: 1 (positivo) si author_rating > 6, 0 (negativo) de lo contrario

In [11]:
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

In [12]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]


Dividir el dataset en entrenamiento y prueba

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    stratify=y_data,  # Mantener la proporción de clases en ambos conjuntos
    random_state=42,
)

## Análisis exploratorio de los datos

para determinar que tipo de limpieza se debe realizar a los datos de texto de `X_train`

In [18]:
X_train.shape  # filas, columnas
# 40000 Reseñas de Peliculas

(40000,)

In [22]:
display(X_train.head())
X_train.info()

32773     \nTercera entrega de las desventuras de Alice ...
28791     \nAntes de ser estudioso del cine y sus técnic...
56725     \nPresupuesto total de la pelicula 300€. Contr...
127226    \nBasada en el libro autobiográfico de Bauby, ...
101456    \nSe nos presentó el film como el primer spin-...
Name: author_review_desc, dtype: object

<class 'pandas.core.series.Series'>
Index: 40000 entries, 32773 to 41947
Series name: author_review_desc
Non-Null Count  Dtype 
--------------  ----- 
40000 non-null  object
dtypes: object(1)
memory usage: 625.0+ KB


## 1) Limpieza de los datos de texto

Tomar los datos de `X_train` y aplicar las funciones de limpieza que considere necesarias

Ayudas:

- Convertir a minúsculas
- Eliminar caracteres especiales y números
- Corregir palabras mal escritas
- etc


In [ ]:
import re
import unicodedata
from nltk.corpus import stopwords

# Obtener stopwords en español
stop_words = set(stopwords.words("spanish"))


def limpiar_texto(texto):
    """
    Función para limpiar texto:
    1. Convierte a minúsculas
    2. Elimina acentos
    3. Elimina caracteres especiales y números
    4. Elimina stopwords
    5. Elimina espacios múltiples

    Args:
        texto (str): Texto a limpiar

    Returns:
        str: Texto limpio
    """
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar acentos
    texto = unicodedata.normalize("NFKD", texto)
    texto = texto.encode("ASCII", "ignore").decode("utf-8")

    # Eliminar caracteres especiales y números
    texto = re.sub(
        r"[^\w\s]", " ", texto
    )  # Reemplaza caracteres especiales por espacio
    texto = re.sub(r"\d+", " ", texto)  # Reemplaza números por espacio

    # Eliminar espacios múltiples y espacios al inicio/final
    texto = re.sub(r"\s+", " ", texto)
    texto = texto.strip()

    # Tokenización simple y eliminación de stopwords
    tokens = texto.split()
    tokens = [token for token in tokens if token not in stop_words]

    # Unir tokens sin stopwords
    texto = " ".join(tokens)

    return texto


# Ejemplo de uso con una reseña
ejemplo_texto = X_train.iloc[0]
print("Texto original:")
print(ejemplo_texto)
print("\nTexto limpio (sin stopwords):")
print(limpiar_texto(ejemplo_texto))

Texto original:

Tercera entrega de las desventuras de Alice contra la corporación Umbrella, si la primera me sorprendió gratamente y la segunda me decepcionó completamente, esta se halla en su justo término medio, aunque ya poco queda de aquella película agobiante y opresiva que me sedujo en el 2.002: el terror y la angustia han sido sustituidos ya completamente por la acción, y donde antes cada zombie era una amenaza ahora sólo vemos una pieza a abatir.


De todas maneras, no se puede negar que la película es entretenida y no baja el listón en ningún momento, manteniendo a base de explosiones y disparos lo que le falta de ambientación y suspense.  Pero tampoco vamos a pedirle peras al olmo, y no hay más que mirar quién la dirige para que nos demos por más que satisfechos con el resultado final (con el señor Mulcahy de por medio, cualquier bodrio es posible).


Para pasar el rato.


Texto limpio (sin stopwords):
tercera entrega desventuras alice corporacion umbrella si primera sorpren

In [30]:
# Aplicar la limpieza a todo X_train
print("Limpiando X_train...")
X_train_limpio = X_train.apply(limpiar_texto)

# Aplicar la limpieza a X_test para mantener consistencia
print("Limpiando X_test...")
X_test_limpio = X_test.apply(limpiar_texto)

# Verificar las dimensiones
print("\nDimensiones de los datos:")
print(f"X_train_limpio: {X_train_limpio.shape}")
print(f"X_test_limpio: {X_test_limpio.shape}")


Limpiando X_train...
Limpiando X_test...
Limpiando X_test...

Dimensiones de los datos:
X_train_limpio: (40000,)
X_test_limpio: (10000,)

Dimensiones de los datos:
X_train_limpio: (40000,)
X_test_limpio: (10000,)


## 2) Representacion del texto

Luego de tener los datos limpios, realizar la representación de los datos de texto para poder usarse en modelos de machine learning.

In [34]:
# Corpus -> X_train_limpio
X_train_limpio.size

40000

In [40]:
cv = CountVectorizer()
bow_matrix = cv.fit_transform(X_train_limpio)

print("Dimensiones de la matriz Bag-of-Words:", bow_matrix.shape)
print(
    "Ejemplo de términos (BoW):",
    np.random.choice(cv.get_feature_names_out(), size=10, replace=False),
)


Dimensiones de la matriz Bag-of-Words: (40000, 134489)
Ejemplo de términos (BoW): ['terminos' 'empeora' 'embriagar' 'realizarlas' 'guerreras' 'krystof'
 'dorman' 'halley' 'averiaba' 'montanes']


In [41]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(X_train_limpio)

print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)
print(
    "Ejemplo de términos (TF-IDF):",
    np.random.choice(tfidf.get_feature_names_out(), size=10, replace=False),
)

Dimensiones de la matriz TF-IDF: (40000, 134489)
Ejemplo de términos (TF-IDF): ['halen' 'otorgada' 'salim' 'impostados' 'disfrutase' 'kaba' 'refugiar'
 'electrical' 'prfsa' 'macu']


In [42]:
# Obtener nombres de términos y TF-IDF promedio por término en todo el corpus
feature_names = tfidf.get_feature_names_out()
mean_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).ravel()

# Normalizar para interpretar como "probabilidades" (suma 1)
total = mean_tfidf.sum()
probs = mean_tfidf / total if total > 0 else np.zeros_like(mean_tfidf)

# DataFrame con término, tfidf medio y probabilidad normalizada
df_terms = pd.DataFrame(
    {
        "term": feature_names,
        "mean_tfidf": mean_tfidf,
        "probability": probs,
    }
)

print(
    "\nEjemplo aleatorio de 10 términos con su probabilidad (TF-IDF medio normalizado):"
)
print(
    df_terms.sample(10, random_state=2025)
    .sort_values("probability", ascending=False)
    .reset_index(drop=True)
)



Ejemplo aleatorio de 10 términos con su probabilidad (TF-IDF medio normalizado):
              term  mean_tfidf   probability
0           puesto    0.002333  2.687405e-04
1          cansina    0.000353  4.062459e-05
2           dawson    0.000130  1.502223e-05
3       anadiremos    0.000015  1.764737e-06
4        catolicas    0.000009  1.011820e-06
5      suburbanita    0.000006  7.468028e-07
6          zandieh    0.000003  3.957730e-07
7      ocultandolo    0.000003  3.494902e-07
8  ransfigurandola    0.000002  2.745553e-07
9      sttepenwolf    0.000002  2.467178e-07


## 3) Entrenar un modelo de machine learning de clasificación

Utilizar un modelo de clasificación para entrenar y evaluar el modelo con los datos preparados.


In [48]:
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(tfidf_matrix, y_train)

,loss,'hinge'
,penalty,'l2'
,alpha,0.0001
,l1_ratio,0.15
,fit_intercept,True
,max_iter,1000
,tol,0.001
,shuffle,True
,verbose,0
,epsilon,0.1
,n_jobs,None


### Evaluar el modelo con los datos de prueba

Usar el modelo para predecir en `X_test` y evaluar con `y_test`

**Nota:** Recuerde que `X_test` debe pasar por los mismos procesos de limpieza y representación que `X_train`

In [51]:
# Usar el transformador TF-IDF entrenado  con lo datos de train
# para procesar los datos de prueba
X_test_processed = tfidf.transform(X_test_limpio)

y_pred = sgd.predict(X_test_processed)

In [52]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.8027
Precision: 0.7980347429373574
Recall: 0.846927374301676
F1 Score: 0.8217544493630861
ROC AUC Score: 0.7991656309953304
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.81      0.75      0.78      4630
           1       0.80      0.85      0.82      5370

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



**Guardar el modelo**

In [62]:
import joblib

joblib.dump(sgd, "../../models/sgd_sentiment_model.pkl")


['../../models/sgd_sentiment_model.pkl']

## Librerías Usadas

In [63]:
from watermark import watermark

print(watermark(python=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.12.11
IPython version      : 9.6.0

joblib   : 1.5.2
re       : 2.2.1
watermark: 2.5.0
nltk     : 3.9.2
pandas   : 2.3.3
numpy    : 2.3.4
sklearn  : 1.7.2



---

## Referencias
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html)
- Ejemplo de Entrenamiento y selección de Modelo de machine learning entre varios modelos <https://joserzapata.github.io/post/ciencia-datos-proyecto-python/6-model_selection/>
- https://joserzapata.github.io/courses/python-ciencia-datos/python/
- https://joserzapata.github.io/courses/python-ciencia-datos/pandas/
- https://joserzapata.github.io/courses/python-ciencia-datos/machine-learning/

Docente: [Jose R. Zapata](https://joserzapata.github.io)

- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)